In [ ]:
from google.colab import files
uploaded = files.upload()


Saving CVC-ClinicDB.rar to CVC-ClinicDB.rar


In [ ]:
!apt-get install unrar
import os
rar_file = "/content/CVC-ClinicDB.rar"
output_dir = "/content/CVC-ClinicDB"


os.makedirs(output_dir, exist_ok=True)


!unrar x -o+ "{rar_file}" "{output_dir}"





Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/CVC-ClinicDB.rar

Creating    /content/CVC-ClinicDB/CVC-ClinicDB                        OK
Creating    /content/CVC-ClinicDB/CVC-ClinicDB/Ground Truth           OK
Extracting  /content/CVC-ClinicDB/CVC-ClinicDB/Ground Truth/1.tif          0%  OK 
Extracting  /content/CVC-ClinicDB/CVC-ClinicDB/Ground Truth/10.tif         0%  OK 
Extracting  /content/CVC-ClinicDB/CVC-ClinicDB/Ground Truth/100.tif        0%  OK 
Extracting  /content/CVC-ClinicDB/CVC-ClinicDB/Ground Truth/101.tif        0%  OK 
Extracting  /content/CVC-ClinicDB/CVC-ClinicDB/Ground Truth/102.tif        0%  OK 
Extracting  /content/CVC-ClinicDB/CVC-ClinicDB/Ground Truth/103.t

In [ ]:
import os
import torch
base_dir = "CVC-ClinicDB/CVC-ClinicDB"
groundTruth_dir = os.path.join(base_dir, "Ground Truth")
original_dir = os.path.join(base_dir, "Original")


if os.path.exists(groundTruth_dir) and os.path.exists(original_dir):
    image_files = os.listdir(groundTruth_dir)
    mask_files = os.listdir(original_dir)

    print(f"Number of images: {len(image_files)}")
    print(f"Number of masks: {len(mask_files)}")
else:
    print(f"Either {groundTruth_dir} or {original_dir} does not exist!")


Number of images: 612
Number of masks: 612


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


base_dir = "CVC-ClinicDB/CVC-ClinicDB"
images_dir = os.path.join(base_dir, "Original")
masks_dir = os.path.join(base_dir, "Ground Truth")


IMG_SIZE = (1024, 1024)
BATCH_SIZE = 16


print("Images directory contents:", os.listdir(images_dir)[:5])
print("Masks directory contents:", os.listdir(masks_dir)[:5])

image_paths = sorted([os.path.join(images_dir, fname) for fname in os.listdir(images_dir) if fname.endswith(".tif")])
mask_paths = sorted([os.path.join(masks_dir, fname) for fname in os.listdir(masks_dir) if fname.endswith(".tif")])


assert len(image_paths) == len(mask_paths), "image not equal to musk！"
print(f"Found {len(image_paths)} images and {len(mask_paths)} masks.")


train_images, val_images, train_masks, val_masks = train_test_split(
    image_paths, mask_paths, test_size=0.2, random_state=42
)
print(f"Training images: {len(train_images)}, Validation images: {len(val_images)}")


class CVCClinicDataset(Dataset):
    def __init__(self, images, masks, transform=None, target_transform=None):
        self.images = images
        self.masks = masks
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):

        image = cv2.imread(self.images[idx], cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.masks[idx], cv2.IMREAD_GRAYSCALE)


        image = cv2.resize(image, IMG_SIZE)
        mask = cv2.resize(mask, IMG_SIZE)


        image = image / 255.0
        mask = (mask > 127).astype(np.float32)


        if self.transform:
            image = self.transform(image).to(torch.float32)
        if self.target_transform:
            mask = self.target_transform(mask).to(torch.float32)

        return image, mask



image_transform = transforms.Compose([
    transforms.ToTensor(),
])

mask_transform = transforms.Compose([
    transforms.ToTensor(),
])


train_dataset = CVCClinicDataset(train_images, train_masks, transform=image_transform, target_transform=mask_transform)
val_dataset = CVCClinicDataset(val_images, val_masks, transform=image_transform, target_transform=mask_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


for images, masks in train_loader:
    print("Image batch shape:", images.shape)
    print("Mask batch shape:", masks.shape)
    break

Images directory contents: ['449.tif', '548.tif', '245.tif', '544.tif', '249.tif']
Masks directory contents: ['449.tif', '548.tif', '245.tif', '544.tif', '249.tif']
Found 612 images and 612 masks.
Training images: 489, Validation images: 123
Image batch shape: torch.Size([16, 3, 1024, 1024])
Mask batch shape: torch.Size([16, 1, 1024, 1024])


In [ ]:
!pip install git+https://github.com/facebookresearch/segment-anything.git
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth -O sam_vit_b.pth


  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-av16ljw1
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-av16ljw1
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36592 sha256=01ee4d68762fe3ea656bb2bb9c76806f004fc7d9614e99cab28db5827d48bf9d
  Stored in directory: /tmp/pip-ephem-wheel-cache-_o4r4g90/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment_anything
--2024-11-07 09:45:07--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.254.124, 108.157.254.121, 108.157.254.102, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|

In [ ]:
from segment_anything import sam_model_registry, SamPredictor


sam_model = sam_model_registry["vit_b"](checkpoint="sam_vit_b.pth")
sam_model.to("cuda" if torch.cuda.is_available() else "cpu")
sam_predictor = SamPredictor(sam_model)

/usr/local/lib/python3.10/dist-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


In [ ]:
def get_bounding_box(mask):
    """
    从真实掩膜中提取边界框 (x_min, y_min, x_max, y_max)
    """
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)
    if not np.any(rows) or not np.any(cols):
        return None
    y_min, y_max = np.where(rows)[0][[0, -1]]
    x_min, x_max = np.where(cols)[0][[0, -1]]
    return x_min, y_min, x_max, y_max


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

def generate_mask_with_box(image, mask):


    bounding_box = get_bounding_box(mask)
    if bounding_box is None:
        print("The mask is empty, skipping...")
        return None

    x_min, y_min, x_max, y_max = bounding_box
    input_box = np.array([[x_min, y_min], [x_max, y_max]])


    sam_predictor.set_image(image)


    masks, _, _ = sam_predictor.predict(box=input_box, multimask_output=False)
    return masks[0]


def evaluate_predictions(pred_mask, true_mask):
    pred_flat = pred_mask.flatten()
    true_flat = true_mask.flatten()
    intersection = np.logical_and(pred_flat, true_flat).sum()
    union = np.logical_or(pred_flat, true_flat).sum()
    iou = intersection / union if union != 0 else 0.0
    precision = precision_score(true_flat, pred_flat, zero_division=0)
    recall = recall_score(true_flat, pred_flat, zero_division=0)
    f1 = f1_score(true_flat, pred_flat, zero_division=0)
    return iou, precision, recall, f1


def visualize_results(image_path, pred_mask, true_mask):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.title("Original Image")
    plt.imshow(image)

    plt.subplot(1, 3, 2)
    plt.title("True Mask")
    plt.imshow(true_mask, cmap="gray")

    plt.subplot(1, 3, 3)
    plt.title("Predicted Mask (Centroid)")
    plt.imshow(pred_mask, cmap="gray")

    plt.show()



In [ ]:
total_iou, total_precision, total_recall, total_f1 = 0.0, 0.0, 0.0, 0.0
num_samples = len(val_images)

for image_path, mask_path in zip(val_images, val_masks):

    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    true_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    true_mask = (true_mask > 127).astype(np.uint8)


    pred_mask = generate_mask_with_box(image, true_mask)
    if pred_mask is None:
        num_samples -= 1
        continue


    iou, precision, recall, f1 = evaluate_predictions(pred_mask, true_mask)
    total_iou += iou
    total_precision += precision
    total_recall += recall
    total_f1 += f1


print(f"Average IoU: {total_iou / num_samples:.4f}")
print(f"Average Precision: {total_precision / num_samples:.4f}")
print(f"Average Recall: {total_recall / num_samples:.4f}")
print(f"Average F1-Score: {total_f1 / num_samples:.4f}")


Average IoU: 0.8055
Average Precision: 0.9244
Average Recall: 0.8582
Average F1-Score: 0.8752
